In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *
from fastai.text import *
from fastai.column_data import *

In [2]:
from fsm import *

In [3]:
DATA_PATH = Path('data')
DATA_PATH
TMP_PATH = DATA_PATH/'tmp'
TMP_PATH.mkdir(exist_ok=True)

In [4]:
sql_ids = np.load(TMP_PATH/'sql_ids.npy')
sql_itos = pickle.load(open(TMP_PATH/'sql_itos.pkl','rb'))
sql_stoi =  collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(sql_itos)})

In [6]:
def get_next_random(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    p = to_np(p)[0]
    pi = p.argsort()[-10:]
    i = np.random.choice(pi, p=softmax_np(p[pi]))
    return indices_char[i]
def softmax_np (nparray):
    ans = np.exp(nparray)
    return ans/ans.sum()

def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    p = to_np(p)
    
    return indices_char[i]

def get_sentence(inp):
    sentence = inp
    counter = 0
    tmp = []
    while "_eos_" not in tmp:   
        tmp = get_next(sentence[-8:]) # always get last three
        sentence += [tmp]
        counter = counter+1
        if counter> 500:
            break
    return sentence
        

def get_sentence_random(inp):
    sentence = inp
    counter = 0
    tmp = []
    while "_eos_" not in tmp:   
        tmp = get_next_random(sentence[-8:]) # always get last three
        sentence += [tmp]
        counter = counter+1
        if counter> 500:
            break
    return sentence
        

def show_sentence(inp):
    sentence =''
    captialize = False
    for i in inp:
        if i =='t_up':
            captialize = True
        else:            
            if captialize:
                sentence = sentence + " " + i.upper()       
           
            else:
                sentence = sentence +" " + i
            captialize = False
    return sentence

In [8]:
fsm = Rule_Parse_FSM()

In [9]:
get_next_random("select userid from users where ".split())

NameError: name 'char_indices' is not defined

In [48]:
get_next_random_fsm("select userid from users where ".split(), fsm)

skip 'location' in STATE: NEW_GROUP


NameError: name 'indice_char' is not defined

## Setup

Set up directory path

In [9]:
dataset = pd.read_json(DATA_PATH/'train.json')
dataset.head()


,comments,description,id,sql,sql_plain,title,url
0,[],,586912,"SELECT p.Title,p.score,p.ViewCount,p.AnswerCou...","SELECT \np.Title,p.score,p.ViewCount,p.AnswerC...",Repent 151-160,http://data.stackexchange.com/stackoverflow/re...
1,[This is test data for my question to make thi...,,54791,"CREATE TABLE #CustomerDB ( ID int NOT NULL, NA...",-- This is test data for my question to make t...,This is test data for my question to make thin...,http://data.stackexchange.com/stackoverflow/re...
2,[],,611565,select Id from Comments where lower(Text) like...,select Id\nfrom Comments \nwhere lower(Text) l...,Accept answer,http://data.stackexchange.com/stackoverflow/re...
3,[],,612812,"DECLARE @From DATETIME = convert(DATETIME, '##...","DECLARE @From DATETIME = convert(DATETIME, '##...",Questions with most views created within 3 mon...,http://data.stackexchange.com/stackoverflow/re...
4,"[Most Prolific Answerers (with score), Shows t...",,325114,"SELECT TOP 50 OwnerUserId as [User Link], COUN...",-- Most Prolific Answerers (with score)\n-- Sh...,Most Prolific Answers,http://data.stackexchange.com/stackoverflow/re...


In [49]:
get_next_random("select userid from users where ".split())

'posts'

In [10]:
SQL = dataset.sql

In [11]:
## Transform the text to be tokenize readily, as certain token has special meaning in SQL, ie. "." means the schema relation
token_replaced = {',' : ' , ',
                 '#' : ' # ',
                 '@' : ' @ ',
                 '!' : ' ! ',
                 '.' : ' . ',
                 '%' : ' % ',
                 '?' : ' ? ',
                 ')' : ' ) ',
                 '(' : ' ( ',
                 '=' : ' = ',
                 '/' : ' / ',
                 '*' : ' * '}
    

In [15]:
def fixup(x):
    for bereplaced, replace in token_replaced.items():        
        x = x.replace(bereplaced, replace).lower() # CASE insentive for SQL
    return x

In [16]:
SQL = SQL.apply(fixup)## Sanity Check
SQL.iloc[0]

'select p . title , p . score , p . viewcount , p . answercount , p . commentcount , len ( p . body )  as ques_size ,  p . favoritecount , p . id ,  u . reputation as user_repo , u . creationdate as user_join_date ,  datediff ( minute ,  p . creationdate ,  a . creationdate )  as qatimegap from posts as p ,  posts as a ,  users as u where p . id = a . parentid and p . owneruserid = u . id and a . creationdate  =   ( select min ( tau . creationdate )  from posts tau where tau . parentid = p . id group by tau . parentid )  and  (  p . title like \' % recommended method for handling unsupportedencodingexception from string . getbytes ( "utf-8" )  % \' or p . title like \' % way to format strings with " ? " parameters to full string in java ?  % \' or p . title like \' % how to parse a string without regular expressions % \' or p . title like \' % how to replace string only once without regex in java ?  % \' or p . title like \' % efficiently removing specific characters  ( some punctuatio

In [17]:
SQL.iloc[0]

'select p . title , p . score , p . viewcount , p . answercount , p . commentcount , len ( p . body )  as ques_size ,  p . favoritecount , p . id ,  u . reputation as user_repo , u . creationdate as user_join_date ,  datediff ( minute ,  p . creationdate ,  a . creationdate )  as qatimegap from posts as p ,  posts as a ,  users as u where p . id = a . parentid and p . owneruserid = u . id and a . creationdate  =   ( select min ( tau . creationdate )  from posts tau where tau . parentid = p . id group by tau . parentid )  and  (  p . title like \' % recommended method for handling unsupportedencodingexception from string . getbytes ( "utf-8" )  % \' or p . title like \' % way to format strings with " ? " parameters to full string in java ?  % \' or p . title like \' % how to parse a string without regular expressions % \' or p . title like \' % how to replace string only once without regex in java ?  % \' or p . title like \' % efficiently removing specific characters  ( some punctuatio

In [4]:
sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL)) 

NameError: name 'SQL' is not defined

In [19]:
sql_tok[0]

['select',
 'p',
 '.',
 'title',
 ',',
 'p',
 '.',
 'score',
 ',',
 'p',
 '.',
 'viewcount',
 ',',
 'p',
 '.',
 'answercount',
 ',',
 'p',
 '.',
 'commentcount',
 ',',
 'len',
 '(',
 'p',
 '.',
 'body',
 ')',
 'as',
 'ques_size',
 ',',
 'p',
 '.',
 'favoritecount',
 ',',
 'p',
 '.',
 'i',
 'd',
 ',',
 'u',
 '.',
 'reputation',
 'as',
 'user_repo',
 ',',
 'u',
 '.',
 'creationdate',
 'as',
 'user_join_date',
 ',',
 'datediff',
 '(',
 'minute',
 ',',
 'p',
 '.',
 'creationdate',
 ',',
 'a',
 '.',
 'creationdate',
 ')',
 'as',
 'qatimegap',
 'from',
 'posts',
 'as',
 'p',
 ',',
 'posts',
 'as',
 'a',
 ',',
 'users',
 'as',
 'u',
 'where',
 'p',
 '.',
 'i',
 'd',
 '=',
 'a',
 '.',
 'parentid',
 'and',
 'p',
 '.',
 'owneruserid',
 '=',
 'u',
 '.',
 'i',
 'd',
 'and',
 'a',
 '.',
 'creationdate',
 '=',
 '(',
 'select',
 'min',
 '(',
 'tau',
 '.',
 'creationdate',
 ')',
 'from',
 'posts',
 'tau',
 'where',
 'tau',
 '.',
 'parentid',
 '=',
 'p',
 '.',
 'i',
 'd',
 'group',
 'by',
 'tau',
 '.',

In [20]:
def toks2ids(tok,pre):
    freq = Counter(p for o in tok for p in o)
    itos = [o for o,c in freq.most_common()]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    ids = np.array([([0] + [stoi[o] for o in p] + [2]) for p in tok]) # pad bos at beginning and eos at the end
    np.save(TMP_PATH/f'{pre}_ids.npy', ids)
    pickle.dump(itos, open(TMP_PATH/f'{pre}_itos.pkl', 'wb'))
    return ids,itos,stoi

In [21]:
sql_ids,sql_itos,sql_stoi = toks2ids(sql_tok,'sql')

In [5]:
sql_ids_concat= []
for i in sql_ids:
    sql_ids_concat += i

In [6]:
sql_ids.shape

(24594,)

In [7]:
vocab_size = len(sql_itos)
print('total chars:', vocab_size)

total chars: 23125


### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [8]:
idx = sql_ids_concat

In [9]:
char_indices = sql_stoi
indices_char = sql_itos

### Create and train model

Pick a size for our hidden state

In [10]:
n_hidden = 256
n_fac = 42

In [22]:
y[:cs]

array([ 5, 18,  4, 33,  5])

## Multi-output model

In [ ]:
cs=5

In [12]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

In [13]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [14]:
xs = np.stack(c_in_dat)
xs.shape

(429356, 5)

In [15]:
ys = np.stack(c_out_dat)
ys.shape

(429356, 5)

In [16]:
xs[:3,:cs]

array([[  0,  13,  18,   4, 102],
       [  5,  18,   4,  33,   5],
       [ 18,   4,  91,   5,  18]])

### Create and train model

In [17]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [19]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=256)

## Stateful model

### GRU

In [20]:
import pdb

In [22]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [60]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, self.h)
        return F.log_softmax(self.l_out(outp), dim=-1)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

* cs is the input , cs = 5 for sequence Length, a tuple with length(cs)
* each tensor is 512, length = bs # batch size
* `(Pdb) torch.stack(cs).size()`
* `torch.Size([5, 512])`
* `(Pdb) outp.size()
torch.Size([5, 512, 256])`
*`(Pdb) !h.size()
torch.Size([1, 512, 256])`
*(Pdb) self.l_out(outp).size()
torch.Size([5, 512, 23125])
*(Pdb) self.l_out
Linear(in_features=256, out_features=23125, bias=True)



In [61]:
# m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))

In [62]:
m = CharSeqStatefulGRU(len(sql_itos), n_fac, 512).cuda()

opt = optim.Adam(m.parameters(), 1e-3)

In [63]:
fit(m, md, 1, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

 83%|████████▎ | 557/671 [00:43<00:08, 12.75it/s, loss=2.76]

KeyboardInterrupt: 

In [29]:
n
n
fit(m, md, 6, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.68409  1.67784]                                 
[ 1.       1.49813  1.52661]                                 
[ 2.       1.41674  1.46769]                                 
[ 3.       1.36359  1.43818]                                 
[ 4.       1.33223  1.41777]                                 
[ 5.       1.30217  1.40511]                                 



In [30]:
set_lrs(opt, 1e-4)

In [31]:
fit(m, md, 3, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.22708  1.36926]                                 
[ 1.       1.21948  1.3696 ]                                 
[ 2.       1.22541  1.36969]                                 



In [18]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [19]:
fit(m, md, 2, lo.opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.72032  1.64016]                                 
[ 1.       1.62891  1.58176]                                 



## Test Sentence

In [ ]:
def get_next_random_fsm(inp, fsm):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    p = to_np(p)[0]
    pi = p.argsort()[-10:]
    i = np.random.choice(pi, p=softmax_np(p[pi]))
    
    if fsm.run(indices_char[i]):
        return indice_char[i]
    else: 
        for pii in pi:
            if fsm.run(indice_char[pii]):
                print('pii loop')
                return indice_char[pii]
            else:
                print('Error, no valid words at all')  
    
    
    return "ERROR"

fsm = Rule_Parse_FSM()

get_next_random("select userid from users where ".split())

get_next_random_fsm("select userid from users where ".split(), fsm)

In [ ]:
a=get_sentence(['select','*','from','where','users'])

b=get_sentence_random(['select','*','from','where','users'])

show_sentence(b)

show_sentence(a)

show_sentence(a)

get_next(['select', '*', 'from'])